In [1]:
import os
import fiona
import rasterio
import rasterio.mask
import rasterio.features
from rasterio.enums import Resampling
import geopandas as gpd
import numpy as np

In [2]:
data_dir = "./data"
output_dir = "./outputs"

dekad_of_interest = "03d2"

start_year = 1989
end_year = 2019

In [3]:
# Get the rainfall data from 1989 t0 2019
data_files = []

for year in range(start_year, end_year + 1):
    file = data_dir + os.sep + \
        f"rfb_blended_moz_dekad/mozrfb{year}{dekad_of_interest}.tif"
    if os.path.exists(file):
        data_files.append(file)

# Test length of files
assert len(data_files) == end_year - start_year + 1

In [4]:
nodata = -1

shape = [len(data_files), 1, 1]

with rasterio.open(data_files[0]) as ds:
    shape[1] = ds.height
    shape[2] = ds.width
    profile = ds.profile

In [5]:
stacked_array = np.empty(shape=shape)
for idx, file in enumerate(data_files):
    with rasterio.open(file) as ds:
        pixArray = ds.read()
        pixArray[pixArray == ds.nodata] = nodata
        stacked_array[idx] = pixArray[0]

array_percentile90 = np.percentile(stacked_array, 95, 0)

In [6]:
# Save the percentile to file
profile.update(nodata=nodata)
with rasterio.open(output_dir + os.sep + 'percentile90.tif', 'w',
                   **profile) as dst:
    dst.write(array_percentile90, 1)

In [7]:
gdf_boundary = gpd.read_file(data_dir + os.sep + "geoBoundaries-MOZ-ADM2.geojson")

# add new attributes to the dataframe

gdf_boundary["percentile_mean"] = None
gdf_boundary["2024_march_mean"] = None
gdf_boundary["exposed_population"] = None
gdf_boundary

,shapeName,shapeISO,shapeID,shapeGroup,shapeType,geometry,percentile_mean,2024_march_mean,exposed_population
0,Alto Molocue,,85939544B55280380491119,MOZ,ADM2,"POLYGON ((37.59634 -15.19208, 37.59621 -15.192...",None,None,None
1,Ancuabe,,85939544B12924931865401,MOZ,ADM2,"POLYGON ((40.02725 -12.84019, 40.02698 -12.837...",None,None,None
2,Angoche,,85939544B28799715012212,MOZ,ADM2,"MULTIPOLYGON (((39.79597 -16.35094, 39.79597 -...",None,None,None
3,Angonia,,85939544B12573256040215,MOZ,ADM2,"POLYGON ((34.51998 -14.68475, 34.51995 -14.684...",None,None,None
4,Balama,,85939544B22682737370097,MOZ,ADM2,"POLYGON ((38.70706 -13.48413, 38.69236 -13.474...",None,None,None
...,...,...,...,...,...,...,...,...,...
154,Tsangano,,85939544B8789366603018,MOZ,ADM2,"POLYGON ((34.06163 -15.14411, 34.06115 -15.144...",None,None,None
155,Vanduzi,,85939544B72355494669336,MOZ,ADM2,"POLYGON ((33.27569 -18.55216, 33.27581 -18.553...",None,None,None
156,Vilankulo,,85939544B70195584825603,MOZ,ADM2,"MULTIPOLYGON (((35.52187 -22.18423, 35.52297 -...",None,None,None
157,Zavala,,85939544B42929876675904,MOZ,ADM2,"POLYGON ((34.42748 -24.46082, 34.42563 -24.462...",None,None,None


In [8]:
# Task 2

with fiona.open(data_dir + os.sep + "geoBoundaries-MOZ-ADM2.geojson", "r") as boundary_ds:
    for feature in boundary_ds:
        with rasterio.open(output_dir + os.sep + 'percentile90.tif') as percentile_ds:
            percentile90_array_masked, out_transform = rasterio.mask.mask(
                percentile_ds, [feature["geometry"]], crop=True)

            percentile90_array_masked[percentile90_array_masked == percentile_ds.nodata] = 0
            gdf_boundary['percentile_mean'] = np.where(gdf_boundary['shapeID'] == feature["properties"]["shapeID"], np.mean(percentile90_array_masked), gdf_boundary['percentile_mean'])

        with rasterio.open(data_dir + os.sep + f"rfb_blended_moz_dekad/mozrfb2024{dekad_of_interest}.tif") as src:
            array_march_2023_masked, out_transform = rasterio.mask.mask(
                src, [feature["geometry"]], crop=True)

            array_march_2023_masked[array_march_2023_masked == src.nodata] = 0
            gdf_boundary['2024_march_mean'] = np.where(gdf_boundary['shapeID'] == feature["properties"]["shapeID"], np.mean(array_march_2023_masked), gdf_boundary['2024_march_mean'])


In [9]:
# Compute if march 2024 exceeds 95 percentile

# Compute per feature
gdf_boundary["exceeds"] = gdf_boundary['2024_march_mean'] > gdf_boundary['percentile_mean']
gdf_boundary.to_file(output_dir + os.sep + "admin_2_computed.shp")


# Compute per pixels

with rasterio.open(data_dir + os.sep + f"rfb_blended_moz_dekad/mozrfb2024{dekad_of_interest}.tif") as src:
    array_march_2023 = src.read()
    binary_array = np.where(array_march_2023 > array_percentile90, 1, 0)
    nodata = src.nodata
    binary_array[array_march_2023 == src.nodata] = src.nodata
    

# Save binary array
profile.update(nodata=nodata)
with rasterio.open(output_dir + os.sep + 'binary.tif', 'w',
                   **profile) as dst:
    dst.write(binary_array[0], 1)

C:\Users\OhuruRO\AppData\Local\Temp\ipykernel_9896\253354515.py:5: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  gdf_boundary.to_file(output_dir + os.sep + "admin_2_computed.shp")


In [10]:
with rasterio.open(data_dir + os.sep + "moz_ppp_2020_UNadj_constrained.tif") as pop_src:
    pop_array = pop_src.read()
    with rasterio.open(output_dir + os.sep + 'binary.tif') as binary_src:
        # resample binary array to population shape        
        binary_array = binary_src.read(
            out_shape=(
                pop_src.count,
                pop_src.height,
                pop_src.width
            ),
            resampling=Resampling.nearest
        )
        profile = binary_src.profile
        pop_exposed = np.where(binary_array != binary_src.nodata, pop_array * binary_array, binary_src.nodata)

In [11]:
# Save exposed population
with rasterio.open(output_dir + os.sep + 'exposed_population.tif', 'w',
                   **profile) as dst:
    dst.write(pop_exposed[0], 1)

In [12]:
exposed_pop_src = rasterio.open(output_dir + os.sep + "exposed_population.tif")
with fiona.open(output_dir + os.sep + "admin_2_computed.shp", "r") as boundary_ds:
    for feature in boundary_ds:
        pixArray, out_transform = rasterio.mask.mask(exposed_pop_src, [feature["geometry"]], crop=True)
        pixArray[pixArray == exposed_pop_src.nodata] = 0
        gdf_boundary['exposed_population'] = np.where(gdf_boundary['shapeID'] == feature["properties"]["shapeID"], np.sum(pixArray), gdf_boundary['exposed_population'])

gdf_boundary.to_file(output_dir + os.sep + "admin_2_computed.shp")

C:\Users\OhuruRO\AppData\Local\Temp\ipykernel_9896\849073553.py:8: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  gdf_boundary.to_file(output_dir + os.sep + "admin_2_computed.shp")
